In [1]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
csvdir = "/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics"

dataframes = []
for csv in os.listdir(csvdir):
    fullpath = os.path.join(csvdir, csv)
    print(fullpath)
    if os.path.isfile(fullpath):
        # Read a csv into a dataframe and append to a list of dataframes
        dataframe = pd.read_table(fullpath)
        dataframes.append(dataframe)

# Concatenate all created dataframes into one
df = pd.concat(dataframes)
df.head()

/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2012.txt
/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2015.txt
/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2004.txt
/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2005.txt
/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2003.txt
/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2014.txt
/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2011.txt
/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2008.txt
/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2007.txt
/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underl

,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths
0,NaN,"Autauga County, AL",1001.0,2012.0,2012.0,All other non-drug and non-alcohol causes,O9,508.0
1,NaN,"Baldwin County, AL",1003.0,2012.0,2012.0,Drug poisonings (overdose) Unintentional (X40-...,D1,24.0
2,NaN,"Baldwin County, AL",1003.0,2012.0,2012.0,All other alcohol-induced causes,A9,20.0
3,NaN,"Baldwin County, AL",1003.0,2012.0,2012.0,All other non-drug and non-alcohol causes,O9,1746.0
4,NaN,"Barbour County, AL",1005.0,2012.0,2012.0,All other non-drug and non-alcohol causes,O9,270.0


In [3]:
# read all original txt files
txt1 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2003.txt')
txt2 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2004.txt')
txt3 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2005.txt')
txt4 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2006.txt')
txt5 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2007.txt')
txt6 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2008.txt')
txt7 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2009.txt')
txt8 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2010.txt')
txt9 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2011.txt')
txt10 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2012.txt')
txt11 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2013.txt')
txt12 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2014.txt')
txt13 = pd.read_table('/workspaces/pds-2022-yellow-team/00_source_data/US_VitalStatistics/Underlying Cause of Death, 2015.txt')

In [4]:
# calculate the total rows of all original txt files
num_rows = 0
for i in range(1,14):
    num_rows += (eval(f'txt{i}').shape[0])

In [5]:
# make sure the number of rows in concatenated dataframe is equal to the total rows of all original txt files
assert df.shape[0] == num_rows

In [6]:
# drop unnecessary column
temp = df.copy()
temp.drop('Notes', axis=1, inplace=True)

In [7]:
# fill missing values with 0
temp.dropna(how='all', inplace=True)
assert temp.isna().sum().sum() == 0

In [8]:
# select cause of death containing drug poisoning
only_od_all_years = temp[temp['Drug/Alcohol Induced Cause'].str.contains('Drug poisonings')]

In [9]:
# split county and state
only_od_all_years[['CTYNAME', 'STNAME']] = only_od_all_years['County'].str.split(',', 1, expand=True)

# remove the space in front of state
only_od_all_years['STNAME'] = only_od_all_years['STNAME'].str.strip()
assert len(only_od_all_years.columns) - len(temp.columns) == 2

In [10]:
# select the states we are interested in
states = ['AZ', 'CO', 'FL', 'HI', 'LA', 'NV', 'NY', 'OK', 'OR', 'SC', 'TX', 'WA', 'WI']
only_od_all_years = only_od_all_years[only_od_all_years['STNAME'].isin(states)]

In [11]:
only_od_all_years['STNAME'].value_counts().sort_index()

AZ    172
CO    215
FL    645
HI     46
LA    205
NV     67
NY    364
OK    169
OR    153
SC    198
TX    537
WA    266
WI    166
Name: STNAME, dtype: int64

In [12]:
# write only_od to csv
only_od_all_years.to_csv('/workspaces/pds-2022-yellow-team/20_intermediate_files/only_od_all_years.csv', index=False)